In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

```markdown
1. Ortam Değişkenlerini Yükleme
Ne işe yarar?: Bu kod, ortam değişkenlerini yüklemek için kullanılır. load_dotenv() fonksiyonu, .env dosyasındaki ortam değişkenlerini (örneğin, API anahtarları) Python ortamına yükler.
Amaç: API anahtarları gibi hassas bilgileri güvenli bir şekilde saklamak ve koda erişmek. Bu durumda, Grok modelini kullanmak için gerekli olan GROQ_API_KEY anahtarını yüklemek için kullanılıyor.
Neden kullanılıyor?: Güvenlik açısından, API anahtarlarını doğrudan koda yazmak yerine bir .env dosyasında saklamak yaygın bir uygulamadır. Bu, kodu paylaşırken hassas bilgilerin ifşa olmasını önler.
```

In [2]:
from langchain_groq import ChatGroq

api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019CDD65FF80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019CDD2CF140>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

```markdown
2. Grok Modelini Yükleme
Ne işe yarar?: Bu kod, LangChain'in Grok entegrasyonunu kullanarak bir dil modeli (LLM) nesnesi oluşturur. ChatGroq sınıfı, xAI'nin Grok modeline bağlanmak için kullanılır.
Amaç: Metin özetleme ve diğer dil işleme görevleri için Grok modelini (gemma2-9b-it) kullanmak. api_key ile kimlik doğrulama yapılır.
Neden kullanılıyor?: Grok, güçlü bir yapay zeka modelidir ve LangChain ile entegre edilerek konuşma özetleme gibi görevler için kullanılabilir. os.getenv ile API anahtarı güvenli bir şekilde alınır.
Detaylar:

    model="gemma2-9b-it": Grok'un belirli bir modeli seçilir.
    groq_api_key: API anahtarı, Grok servislerine erişim sağlar.

Çıktı: ChatGroq nesnesinin bir temsilini döndürür, modelin ve yapılandırmanın detaylarını içerir.
```

In [3]:
from langchain.schema import(
    AIMessage,
    HumanMessage,SystemMessage
)

```markdown
3. LangChain Mesaj Şemalarını İçe Aktarma

Ne işe yarar?: LangChain'in mesaj sınıflarını (AIMessage, HumanMessage, SystemMessage) içe aktarır. Bu sınıflar, sohbet tabanlı bir modelle iletişim kurmak için kullanılır.
    Amaç: Modelle etkileşim kurarken farklı roller (sistem, insan, yapay zeka) için mesaj türlerini tanımlamak.
    Neden kullanılıyor?: LangChain, sohbet modelleriyle çalışırken mesajların rollerini açıkça belirtmeyi gerektirir. Örneğin:
        SystemMessage: Modelin davranışını tanımlayan talimatlar (örn. "Özetleme konusunda uzman bir assistantsın").
        HumanMessage: Kullanıcının modele gönderdiği istek veya soru.
        AIMessage: Modelin verdiği yanıt.

Bu, daha sonra konuşma özetleme için kullanılacak mesajları yapılandırmak için gerekli.
```

In [4]:
speech = """
People across the country, involved in government, political, and social activities, are dedicating their time to make the ‘Viksit Bharat Sankalp Yatra' 
(Developed India Resolution Journey) successful. Therefore, as a Member of Parliament, it was my responsibility to also contribute my time to this program. 
So, today, I have come here just as a Member of Parliament and your ‘sevak’, ready to participate in this program, much like you.

In our country, governments have come and gone, numerous schemes have been formulated, discussions have taken place, and big promises have been made.
However, my experience and observations led me to believe that the most critical aspect that requires attention is ensuring that the government's plans
reach the intended beneficiaries without any hassles. If there is a ‘Pradhan Mantri Awas Yojana’ (Prime Minister's housing scheme), then those who are living in
jhuggis and slums should get their houses. And he should not need to make rounds of the government offices for this purpose. The government should reach him. 
Since you have assigned this responsibility to me, about four crore families have got their ‘pucca’ houses. However, I have encountered cases where someone is
left out of the government benefits. Therefore, I have decided to tour the country again, to listen to people's experiences with government schemes, 
to understand whether they received the intended benefits, and to ensure that the programs are reaching everyone as planned without paying any bribes. 
We will get the real picture if we visit them again. Therefore, this ‘Viksit Bharat Sankalp Yatra' is, in a way, my own examination. 
I want to hear from you and the people across the country whether what I envisioned and the work I have been doing aligns with reality and whether
it has reached those for whom it was meant. 
"""

```markdown
4. Örnek Konuşma Metnini Tanımlama
Ne işe yarar?: Bir konuşma metni (speech) tanımlanır. Bu metin, Hindistan'da bir milletvekilinin "Viksit Bharat Sankalp Yatra" programına katılımını ve hükümet programlarının hedef kitlelere ulaştırılmasının önemini anlattığı bir konuşmadır.
Amaç: Bu metin, özetleme görevleri için kullanılacak örnek bir giriş verisi olarak tanımlanır.
Neden kullanılıyor?: Özetleme algoritmalarını test etmek ve modelin metni nasıl özetlediğini görmek için bir örnek metin gerekiyor. Bu konuşma, daha sonra farklı özetleme yöntemleriyle işlenecek.
```

In [5]:
chat_message = [SystemMessage(content="You are expertise in summarizing speeched"),
                HumanMessage(content=f"Please provide a short and concise summary of following speech:\n Text: {speech}")]

```markdown
5. İlk Özetleme Denemesi: Doğrudan Mesajla
Ne işe yarar?: Modelle iletişim kurmak için bir sohbet mesajı listesi oluşturulur. SystemMessage ile modelin özetleme konusunda uzman olduğu belirtilir, HumanMessage ile özetlenmesi istenen konuşma metni gönderilir.
Amaç: Modelden konuşmanın kısa ve öz bir özetini almak.
Neden kullanılıyor?: Bu, en basit özetleme yöntemidir. Doğrudan modelle konuşarak özetleme yapılır, herhangi bir zincir (chain) kullanılmaz. Bu yöntem, küçük metinler için hızlı bir testtir.
```

In [6]:
llm.get_num_tokens(speech)

416

```markdown
6. Token Sayısını Kontrol Etme
Ne işe yarar?: Konuşma metninin (speech) token sayısını hesaplar.
Amaç: Modelin giriş sınırlarını kontrol etmek. Dil modelleri genellikle token sayısı sınırına sahiptir (örneğin, 1024 token).
Neden kullanılıyor?: Metnin modelin işleyebileceği boyutta olup olmadığını anlamak için. Çıktı, 416 token olduğunu gösteriyor, bu da modelin sınırları içinde.
Çıktı: 416 (metnin token sayısı).
```

In [7]:
llm(chat_message).content

C:\Users\murat\AppData\Local\Temp\ipykernel_26040\1339791998.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm(chat_message).content


'The speaker, a Member of Parliament, is embarking on the "Viksit Bharat Sankalp Yatra" to assess the effectiveness of government schemes in reaching the intended beneficiaries. \n\nThey acknowledge past efforts but emphasize the need to ensure that programs like the Prime Minister\'s Housing Scheme are delivered smoothly and without corruption.  \n\nThe speaker will personally travel the country, listening to people\'s experiences and verifying if government benefits are reaching everyone as intended. This journey serves as a personal audit to ensure their vision and work align with reality on the ground.  \n\n'

```markdown
7. Özetleme Çıktısını Alma
Ne işe yarar?: chat_message listesindeki mesajları modele gönderir ve özetlenmiş metni alır.
Amaç: Konuşmanın kısa bir özetini oluşturmak.
Neden kullanılıyor?: Bu, doğrudan modelle iletişim kurarak özetleme yapmanın sonucunu gösterir. Çıktı, konuşmanın ana noktalarını (örneğin, milletvekilinin programı, hükümet planlarının ulaşılabilirliği) özetler.
Not: Kodda bir deprecation uyarısı görülüyor (BaseChatModel.__call__ yerine invoke kullanılması öneriliyor). Bu, LangChain'in eski bir sürümünün kullanıldığını gösteriyor.
Çıktı: Konuşmanın kısa bir özeti (örneğin, programın amacı, hükümet planlarının ulaşımı, yolsuzluk olmadan hizmet).
```

In [8]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

generic_template = """
Write a summary of the following speech:
Speech:{speech}
Translate the precise summary to {language}
"""
prompt = PromptTemplate(
    input_variables = ['speech','language'],
    template = generic_template
)


```markdown
8. Genel Özetleme Şablonu Tanımlama
Ne işe yarar?: Bir PromptTemplate oluşturulur. Bu şablon, konuşmayı özetlemeyi ve özeti belirtilen bir dile çevirmeyi talep eder.
    Amaç: Daha yapılandırılmış bir özetleme işlemi için bir şablon tanımlamak. speech ve language değişkenleri, dinamik olarak doldurulabilir.
    Neden kullanılıyor?: PromptTemplate, tekrar kullanılabilir ve özelleştirilebilir bir prompt oluşturmak için kullanılır. Bu, özetleme işlemini farklı diller için genelleştirir (örneğin, Türkçe).
    Detaylar:
        input_variables: Şablonda kullanılacak değişkenler (speech ve language).
        template: Modelin izleyeceği talimatları içerir.
```

In [9]:
complete_prompt = prompt.format(speech=speech,language="Turkish")

```markdown
9. Prompt'u Formatlama
Ne işe yarar?: PromptTemplate'ı kullanarak konuşma metni ve dil (Türkçe) ile formatlanmış bir prompt oluşturur.
Amaç: Modelin konuşmayı özetlemesini ve özeti Türkçeye çevirmesini sağlamak için hazır bir prompt hazırlamak.
Neden kullanılıyor?: Bu, şablonun dinamik olarak doldurulmasını sağlar ve modeli çalıştırmadan önce prompt'un doğru formatta olduğunu kontrol eder.
```

In [10]:
llm.get_num_tokens(complete_prompt)

438

```markdown
10. Token Sayısını Kontrol Etme (Tekrar)
ne işe yarar?: Formatlanmış prompt'un token sayısını hesaplar.
Amaç: Modelin giriş sınırlarına uygun olup olmadığını kontrol etmek.
Neden kullanılıyor?: Özetleme ve çeviri talimatlarının eklenmesiyle token sayısı artar (416'dan 438'e). Bu, modelin sınırlarını aşmadığından emin olmak için yapılır.
Çıktı: 438 (formatlanmış prompt'un token sayısı).
```

In [11]:
llm_chain = LLMChain(llm=llm,prompt=prompt)
summary = llm_chain.run({'speech':speech,'language':'Turkish'})
summary

C:\Users\murat\AppData\Local\Temp\ipykernel_26040\3844134357.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm,prompt=prompt)
C:\Users\murat\AppData\Local\Temp\ipykernel_26040\3844134357.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = llm_chain.run({'speech':speech,'language':'Turkish'})


'Konuşmada, konuşmacı "Viksit Bharat Sankalp Yatra" (Gelişmiş Hindistan Niyet Yolu)\'na katılıp halkın deneyimlerini dinlemek için ülkeyi ziyaret etmektedir. \n\nKonuşmacı, hükümet programlarının hedef kitlelere ulaştırılmasının kritik öneme sahip olduğunu belirterek, "Pradhan Mantri Awas Yojana" (Başbakan Barınma Projesi) gibi programların doğrudan ihtiyaç sahiplerine ulaştırılmasının önemine vurgu yapmaktadır. \n\nDaha önce de dört milyon aileye "pucca" ev dağıtmayı başardıklarını belirtirken, bazı kişilerin hala programlardan yararlanamadığını fark ettiklerini ve bu durumu düzeltmek için ülke genelinde gezmek istediklerini ifade eder. \n\nKonuşmacı, "Viksit Bharat Sankalp Yatra"yı kendi değerlendirmesi olarak nitelendirerek, halktan geri bildirim istemektedir.\n\n---\n\n**Turkish Summary:**\n\nKonuşmada, konuşmacı "Viksit Bharat Sankalp Yatra" adlı program kapsamında ülkeyi ziyaret ederek halkın hükümet programlarına dair deneyimlerini dinlemeyi amaçlıyor. \n\nKonuşmacı, hükümet pro

```markdown
11. LLMChain ile Özetleme ve Çeviri
Ne işe yarar?: LLMChain kullanılarak konuşma özetlenir ve özet Türkçeye çevrilir.
Amaç: Daha yapılandırılmış bir zincir kullanarak özetleme ve çeviri işlemini gerçekleştirmek.
Neden kullanılıyor?: LLMChain, prompt ve modeli birleştirerek tekrar kullanılabilir bir iş akışı sağlar. Bu, özellikle birden fazla özetleme işlemi için kullanışlıdır.
Detaylar:

    llm_chain = LLMChain(llm=llm, prompt=prompt): Model ve prompt'u bir zincire bağlar.
    run({'speech': speech, 'language': 'Turkish'}): Zinciri çalıştırır ve özet ile çeviriyi döndürür.

Not: Yine deprecation uyarısı görülüyor (LLMChain ve run yerine RunnableSequence ve invoke öneriliyor).
Çıktı: Konuşmanın İngilizce özeti ve ardından Türkçeye çevrilmiş hali.
```

In [14]:
## StuffDocumentChain Text summarization
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
docs


[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

```markdown
12. PDF Dosyasından Metin Yükleme
Ne işe yarar?: PyPDFLoader ile bir PDF dosyasından (apjspeech.pdf) metin yüklenir ve belgeler ayrıştırılır.
Amaç: A.P.J. Abdul Kalam'ın konuşmasını içeren bir PDF dosyasını işlemek ve özetlemek.
Neden kullanılıyor?: PDF dosyaları, uzun metinler içerebilir ve bunları doğrudan işlemek yerine LangChain'in belge yükleme araçlarıyla yapılandırılmış bir şekilde işlemek daha kolaydır.
Detaylar:

    PyPDFLoader: PDF'yi sayfa sayfa yükler ve her sayfayı bir Document nesnesi olarak döndürür.
    load_and_split(): PDF'yi yükler ve metni ayrıştırır.

Çıktı: Her biri bir PDF sayfasını temsil eden Document nesnelerinden oluşan bir liste (docs).
```

In [15]:
template = """
Write a concise and short summary of the following speech
Speech :{text}
"""
prompt = PromptTemplate(input_variables=['text'],template=template)

```markdown
13. StuffDocumentsChain için Şablon Tanımlama
Ne işe yarar?: StuffDocumentsChain için bir özetleme şablonu tanımlanır.
Amaç: PDF'den yüklenen belgeleri özetlemek için bir prompt oluşturmak.
Neden kullanılıyor?: StuffDocumentsChain, tüm belgeleri tek bir metin olarak birleştirip özetlemek için kullanılır. Bu şablon, özetleme işlemini yönlendirmek için gerekli.
```

In [16]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm,chain_type="stuff",prompt=prompt,verbose=True)
output_summary = chain.run(docs)
output_summary



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise and short summary of the following speech
Speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online int

'In his departing speech, A.P.J. Abdul Kalam reflects on his five years as President of India, highlighting key experiences and messages he encountered. He emphasizes the aspirations of India\'s youth for a developed nation by 2020, urging them to work towards this goal.\n\nKalam shares stories from his travels, showcasing the resilience and initiative of rural communities, the innovative spirit of individuals, and the unwavering dedication of India\'s defense forces. He stresses the importance of empowering villages, fostering rural development, and harnessing the potential of technological advancements like e-connectivity for societal transformation.\n\nHe calls for a collective effort to eradicate poverty, illiteracy, and societal ills while preserving India\'s cultural heritage. Kalam\'s speech is a testament to his optimism, his belief in India\'s potential, and his unwavering commitment to the nation\'s progress. He urges citizens to work together, driven by the spirit of "we can

```markdown
14. StuffDocumentsChain ile Özetleme
Ne işe yarar?: StuffDocumentsChain kullanılarak tüm belgeler (docs) birleştirilir ve tek bir özet oluşturulur.
Amaç: PDF'deki konuşmanın tamamını (tüm sayfalar) tek bir özet haline getirmek.
Neden kullanılıyor?: StuffDocumentsChain, küçük ve orta boyuttaki belgeler için uygundur çünkü tüm metni tek bir prompt'a sıkıştırır. Ancak, büyük metinler için token sınırlarını aşabilir.
Detaylar:

    StuffDocumentsChain: Belgeleri birleştirir ve prompt ile modele gönderir.
    verbose=True: İşlemin detaylarını gösterir (örneğin, hangi prompt kullanıldı).

Çıktı: PDF'deki konuşmanın kısa bir özeti.
```

In [22]:
# Map Reduce to summarize large documents
from langchain_text_splitters import RecursiveCharacterTextSplitter


final_documents = RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

```markdown
15. Metni Parçalara Ayırma (MapReduce için)
Ne işe yarar?: PDF'den yüklenen belgeleri daha küçük parçalara böler.
Amaç: Büyük metinleri MapReduceDocumentsChain ile işlemek için uygun boyutta parçalara ayırmak.
Neden kullanılıyor?: Büyük metinler, modelin token sınırlarını aşabilir. MapReduce yaklaşımı, metni parçalara ayırarak her birini ayrı ayrı özetler ve ardından özetleri birleştirir.
Detaylar:

    chunk_size=2000: Her parça maksimum 2000 karakter içerir.
    chunk_overlap=100: Parçalar arasında 100 karakterlik bir örtüşme bırakılır, böylece bağlam kaybolmaz.

Çıktı: Her biri yaklaşık 2000 karakterlik Document nesnelerinden oluşan bir liste.
```

In [28]:
chunk_prompt = """
Please summarize the below speech:
Speech: {text}
Summary:
"""
map_prompt_template  = PromptTemplate(input_variables=['text'],template=chunk_prompt)

```markdown
16. MapReduce için Prompt Tanımlama
Ne işe yarar?:

    map_prompt_template: Her bir metin parçasını özetlemek için kullanılan şablon.
    final_prompt_template: Tüm parça özetlerini birleştirerek nihai özeti oluşturmak için kullanılan şablon.

Amaç: MapReduce zincirinde kullanılacak iki aşamalı prompt'lar tanımlamak. İlk aşama (map) her parçayı özetler, ikinci aşama (reduce) bu özetleri birleştirir.
Neden kullanılıyor?: MapReduce, büyük metinleri işlemek için idealdir. İlk prompt, parçaları özetler; ikinci prompt, özetleri birleştirerek yapılandırılmış bir nihai özet üretir.
```

In [29]:
final_prompt = """
Provide the final summary of the entire speech with these important points.
Add a motivation Title,Start the precise summary with an introduction and provide the summary in number of points for the speech.
Speech:{text} 
"""
final_prompt_template = PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a motivation Title,Start the precise summary with an introduction and provide the summary in number of points for the speech.\nSpeech:{text} \n')

In [30]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    verbose=True,
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,  
)
output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have m

Token indices sequence length is longer than the specified maximum sequence length for this model (2651 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a motivation Title,Start the precise summary with an introduction and provide the summary in number of points for the speech.
Speech:In his departing speech, A.P.J. Abdul Kalam reflects on his five years as President of India, expressing gratitude for the experiences and interactions he's had with people from all walks of life. 

He highlights ten key messages from his time in office, emphasizing the importance of:

* **Accelerated development driven by youth aspirations**: He recalls a young girl's question about India's development timeline, highlighting the importance of addressing youth aspirations for a developed nation.
* **Empowering villages**:  He stresses the need to strengthen rural communities.
* **Mobilizing rural talent**:  Kalam believes in harnessing the potential of rural India for national competitiveness.
* **

'##  A Vision for a Developed India: A Summary of A.P.J. Abdul Kalam\'s Final Speech\n\n**Introduction:** \n\nIn his heartfelt farewell address,  A.P.J. Abdul Kalam reflects on his tenure as President of India, expressing gratitude for the experiences and interactions that have shaped his presidency. Beyond reminiscing, Kalam delivers a powerful call to action, outlining ten key messages and a vision for a developed India by 2020.\n\n**Key Points:**\n\n1. **Accelerated Development Driven by Youth Aspirations:** Kalam emphasizes the need to address the aspirations of India\'s youth for a developed nation, citing a young girl\'s question about the timeline for achieving this goal.\n2. **Empowering Villages:** Strengthening rural communities through infrastructure development, financial autonomy for village councils, and better connectivity is crucial for national competitiveness.\n3. **Overcoming Challenges:** Kalam encourages resilience and the ability to succeed despite difficulties, d

```markdown
17. MapReduce ile Özetleme
Ne işe yarar?: MapReduceDocumentsChain kullanılarak metin parçaları ayrı ayrı özetlenir ve ardından bu özetler birleştirilir.
Amaç: Büyük bir konuşma metnini (örneğin, PDF'deki konuşma) özetlemek ve yapılandırılmış bir nihai özet üretmek.
Neden kullanılıyor?: MapReduce, büyük metinler için uygun bir yöntemdir çünkü metni parçalara ayırarak token sınırlarını aşmayı önler ve daha ayrıntılı bir özetleme sağlar.
Detaylar:

    chain_type="map_reduce": Özetleme için MapReduce yöntemini kullanır.
    verbose=True: İşlemin detaylarını gösterir (her parça için prompt ve özet).
    Hata Uyarısı: Kodda bir token hatası görülüyor (Token indices sequence length is longer than the specified maximum sequence length for this model (2651 > 1024)). Bu, bazı metin parçalarının modelin token sınırını aştığını gösteriyor. Çözüm olarak, chunk_size daha küçük bir değere (örneğin, 1000) ayarlanabilir.

Çıktı: Konuşmanın nihai özeti, bir başlık, giriş ve numaralı noktalar şeklinde yapılandırılmış olarak döner.
```

In [31]:
# Refine summarization
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)
output_summary = chain.run(final_documents)
output_summary



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'Here\'s a refined summary incorporating the new context:\n\nIn his farewell speech, A.P.J. Abdul Kalam reflects on his five years as President of India, expressing gratitude for the diverse experiences he shared with people from all walks of life. He highlights ten key messages from his interactions: accelerating development fueled by youth aspirations, empowering villages, leveraging rural skills, prioritizing agricultural growth, overcoming challenges through collaboration, courage in facing calamities, connectivity for societal change, national defense, a youth movement for a developed India by 2020, and a nation that is proud of its leadership and one of the best places to live. \n\nKalam emphasizes the immense motivation within India\'s youth to see their nation develop, sharing their desires for a prosperous, safe, and proud India. This is exemplified by a poignant anecdote about a young girl who questioned why India couldn\'t become developed before 2020, highlighting the urgen

```markdown
18. Refine ile Özetleme
Ne işe yarar?: Refine zinciri, belgeleri sırayla işler ve her adımda özeti iyileştirir.
Amaç: Daha ayrıntılı ve bağlamsal olarak tutarlı bir özet oluşturmak.
Neden kullanılıyor?: Refine, MapReduce'e alternatif bir yöntemdir ve her belge parçasını işledikten sonra önceki özeti geliştirir. Bu, daha tutarlı bir özet sağlayabilir.
```

```markdown
Genel Amaç ve Yapı

    Dosyanın Genel Amacı: Bu notebook, LangChain kütüphanesini kullanarak metin özetleme tekniklerini (doğrudan modelle özetleme, LLMChain, StuffDocumentsChain, MapReduceDocumentsChain, Refine) göstermeyi amaçlar. Ayrıca, özetlerin Türkçeye çevrilmesi gibi ek işlevler içerir.
    Yapı:
        Hazırlık: Ortam değişkenleri ve model yüklenir.
        Basit Özetleme: Küçük bir konuşma metni doğrudan modelle özetlenir.
        Çeviri ile Özetleme: LLMChain kullanılarak özetleme ve çeviri yapılır.
        PDF İşleme: Büyük bir PDF dosyasından metin yüklenir ve farklı zincirlerle özetlenir.
        İleri Teknikler: MapReduce ve Refine ile büyük metinler işlenir.
```